In [ ]:
import os
from PIL import Image
import numpy as np
import random
import tqdm
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.preprocessing.image import img_to_array, load_img

In [ ]:
# Define the root directory where the images are stored.
root_dir = 'Leaf_Images/'
train_dir = 'Leaf_Images/train'
test_dir = 'Leaf_Images/test'

# Create directories if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

In [ ]:
# Disease categories (numbers as strings since they're part of the path)
diseases = ['1', '3', '4', '5', '6', '7', '8']

for label in diseases:
    os.makedirs(f'{train_dir}/{label}', exist_ok = True)
    os.makedirs(f'{test_dir}/{label}', exist_ok = True)

# Loop through each disease directory
for disease in tqdm.tqdm(diseases):
    disease_dir = os.path.join(root_dir, disease)
    
    # List all images in the directory
    for image_name in os.listdir(disease_dir):
        split = random.random()
        if split > 0.2:
            os.rename(f'{disease_dir}/{image_name}', f'{train_dir}/{disease}/{image_name}')
        else:
            os.rename(f'{disease_dir}/{image_name}', f'{test_dir}/{disease}/{image_name}')     

In [ ]:
def create_datagen():
    # Initialize the ImageDataGenerator with the some augmentations
    datagen = ImageDataGenerator(
        # Randomly rotate images in the range (degrees, 0 to 180)
        rotation_range=40,
        # Randomly shear images
        shear_range=0.2,
        # Randomly zoom images
        zoom_range=0.2,
        # Randomly shift images horizontally
        width_shift_range=0.2,
        # Randomly shift images vertically
        height_shift_range=0.2,
        # Randomly flip images horizontally
        horizontal_flip=True,
        # Randomly flip images vertically
        vertical_flip=True,
        # Random brightness changes
        brightness_range=(0.5, 1.5),
        # Ensure input images are scaled to [0, 1]
        rescale=1./255,
    )
    return datagen

def load_and_augment_data(train_dir, test_dir, batch_size=64): #batch_size was 32... 1024 took too long to run with my GPU
    train_datagen = create_datagen()
    test_datagen = ImageDataGenerator(rescale=1./255)  # Only rescaling for validation/test data
    
    # Load and iterate training dataset
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        batch_size=batch_size,
        class_mode='categorical' 
    )
    
    # Load and iterate test dataset
    test_generator = test_datagen.flow_from_directory(
        test_dir,
        batch_size=batch_size,
        class_mode='categorical'
    )
    
    return train_generator, test_generator

train_generator, test_generator = load_and_augment_data(train_dir, test_dir)

In [ ]:
print("TensorFlow version:", tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

if tf.config.list_physical_devices('GPU'):
    print("TensorFlow will automatically use the GPU when available.")
else:
    print("Please check your system configuration.")


'''gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Restrict TensorFlow to only allocate 1GB=1024 of memory on the first GPU
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=18432)] #18GB
        )
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)
'''

In [ ]:
'''
# Generate augmented images batch by batch using the custom generator
num_batches = 1  # Define the number of batches to generate
for i, (X_batch, y_batch) in enumerate(train_generator):
    # Display the augmented images
    for j in range(len(X_batch)):
        plt.imshow(X_batch[j])
        plt.title(f"Augmented Image - Class: {np.argmax(y_batch[j])}")
        plt.axis('off')
        plt.show() 
    if i == num_batches - 1:
        break  # Stop generating batches after reaching the specified number
'''

policy = tf.keras.mixed_precision.Policy('mixed_float16')
tf.keras.mixed_precision.set_global_policy(policy)

# Build CNN model
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(256, 256, 3)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dense(7, activation='softmax', dtype='float32')  # Ensure the final layer uses float32 for stability
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(train_generator, steps_per_epoch=100, epochs=50, validation_data=test_generator, validation_steps=10)

In [ ]:
model.save('my_model_88acc')
model.save('my_model_88acc.h5')